## Lecture 02  
## Python Performance Tips  
### 01/29/2025

## Useful resources
- http://alberto.bietti.me/python-performance-tips/

- https://wiki.python.org/moin/PythonSpeed/PerformanceTips

**Why is Python slower than compiled languages like C and C++?**
- Python is a **dynamic interpreted** language (not a compiled language)

- Dynamic: Types of variables, function arguments, etc. are not known until the program runs

- It is not compiled to the native object code and executed on a computer system

- Dynamic interpreted languages have great flexibility, but suffer significant performance limitations

   - why? 
   - **Difficult to optimize**: The first is due to the dynamic nature of the language. Because the types of Python objects (variables, function arguments, etc.) are not known until the program is run, it is very difficult to optimize the interpreter, since it is not possible to know what the bytecode is going to do before it is executed. 
   - **Dependence on the interpreter**: The second issue is that the bytecode generated from compiling Python is targeted at a Python Virtual Machine (PVM), not at the hardware that the program is running on. The Python interpreter must convert the PVM instructions into hardware instructions in order to execute the bytecode. This coversion is always going to carry some overhead. 
   
---

- Python is easy to learn, write, read, debug

- A large library of built-in functions and libraries: https://docs.python.org/3/library/functions.html

---


### How to optimize?

- Get the program to give correct results

- Then rerun to see if the correct program is slow

- Profile to find which parts of the program consume most of the time 

- Repeat

### Today's topics: 

- There are a variety of techniques that can be used to improve performance: 

- Built-in functions

- Function Call Overhead

- Function Decorator

- Loops, and built-in operators

- Membership operator **in** 


## Timing Python Code

- How can I tell how long my python code takes to run?

In [1]:
# manually to time:

import time

start_time = time.time()

#factorial 500! = 1 * 2 * 3 * ... * 500
fact = 1
for i in range(1, 500): 
    fact *= i

end_time = time.time()

#print(fact)

In [21]:
print("run_time: %f" % (end_time - start_time))

run_time: 0.000927


In [14]:
import time
print(time.time()) # method of Time module is used to get the time in seconds since epoch.
# epoch: ( January 1, 1970, 00:00:00 (UTC))

1706491113.8115108


In [1]:
# Timing Python Code

# timeit module: measuring the execution time of code.

# To see how long it takes a program to run once;

# on average over a bunch of runs, e.g. over k=10000 runs;

import timeit

def my_function():
    fact = 1
    for i in range(1, 500): 
        fact *= i


k = 10000 # number of times run the code
print("run_time:", timeit.timeit(my_function, number=k)) # K run
print("run_time:", timeit.timeit(my_function, number=k) / k) # each run

run_time: 0.6804894999950193
run_time: 6.353062999987742e-05


In [23]:
#if using IPython
# %timeit: Time execution of a Python statement or expression
# %timeit my_function()

%timeit -n 1000 -r 7 my_function()

66.2 µs ± 2.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:


%timeit -r 10 my_function()

%timeit -r 10 -n 1000 my_function()



61.4 µs ± 1.64 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)
61.7 µs ± 2.64 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [2]:
def f(x):
    return x**2

def g(x):
    return x**4

def h(x):
    return x**8


%timeit -n 10000 f(5)

%timeit -n 10000 g(5)

%timeit -n 10000 h(5)


92.2 ns ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
107 ns ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
136 ns ± 24.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Built-in Functions


- One of the easiest ways to improve Python performance is not to execute any Python code at all! 

- Python provides a large number of built-in functions that perform a wide variety of operations. 

- These built-in functions are written in C, and so are generally very fast. 

- See the Python documentation for a list of the available functions: https://docs.python.org/3/library/functions.html


In [28]:

# my_min: takes a list of integers as an argument and finds the minimum value.
def my_min(values):
    min_value = values[0]
    for v in values:
        if v < min_value:
            min_value = v
    return min_value

import random
random_numbers = [random.random() for _ in range(0,100_000)]

print(my_min(random_numbers), min(random_numbers))

#time "my_min()"
%timeit -n 100 my_min(random_numbers)

# However, Python already provides a min function that can be used instead
%timeit -n 100 min(random_numbers)


9.928349441135076e-06 9.928349441135076e-06
2.1 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.39 ms ± 90.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Function Call Overhead  


**How functions affect a program’s performance?**  

- Function call overhead in Python is relatively high, especially compared with the execution speed of builtin functions. 

- The overhead is due, e.g., to loading function objects, loading and checking function arguments, dynamic type checking of function arguments that must be performed before and after the function call.

- One idea is to minimize the number of function calls by handling aggregates


In [29]:
total_sum = 0

def inner(i):
    global total_sum
    total_sum += i
# call inner for each element 
def outer_1():
    for i in range(10_000 + 1): 
        inner(i)
outer_1()        
total_sum

50005000

In [30]:
%%timeit -n 100

total_sum = 0
def inner(i):
    global total_sum
    total_sum += i
    
# The sum of the first n non-negative integers
# S_{n} = 1 + ... + n 

def outer_1():
    for i in range(10_000 + 1): 
        inner(i)
        
outer_1()

1.15 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


- The "inner" function was called 10000 times. 

- Instead, move the loop inside the "aggregate" function and call it only once!

In [61]:
# %%timeit -n 100

def aggregate(l):
    x = 0
    for i in l:
        x = x + i
    return x

def outer_2():
    return aggregate(range(10_000 + 1))
    
%timeit -n 100 outer_2()
print(outer_2())

426 µs ± 84.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
50005000


## Membership Testing

- Python provides the **in** operator (a membership operator) to check if an element exists in a collection. 

- The **in** operator is very fast at checking if an element exists in a **dict** or a **set**, because both dict and set are implemented using a **hash table**. 


In [2]:
letters = 'abcdefghijklmnopqrstuvwxyz'
letters_list = [x + y + z for x in letters for y in letters for z in letters]

print("first 10 members:", letters_list[:10])
print("last  10 members:", letters_list[-10:])

first 10 members: ['aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'aai', 'aaj']
last  10 members: ['zzq', 'zzr', 'zzs', 'zzt', 'zzu', 'zzv', 'zzw', 'zzx', 'zzy', 'zzz']


In [3]:
len(letters_list)

17576

In [4]:
print("len_letters_list = %d" % len(letters_list))
print("len_letters ** 3 = %d = %d ** 3" % (len(letters) ** 3, len(letters)))

len_letters_list = 17576
len_letters ** 3 = 17576 = 26 ** 3


In [29]:
"aaa" in letters_list

True

In [30]:
"zzz" in letters_list

True

In [11]:
#Membership of a List
%timeit ("aaa" in letters_list)
%timeit ("mmm" in letters_list)
%timeit ("zzz" in letters_list)
#-n 100

39.1 ns ± 2.22 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
128 µs ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
245 µs ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The number of loops in %timeit differs depending on how long the operation takes:
-  %timeit dynamically determines the number of loops it needs to run based on how fast the operation is.
- If the operation is very fast (e.g., "aaa" in letters_list), %timeit will perform more loops to get an accurate timing (like thousands or millions of loops).
- If the operation is slower (e.g., "mmm" in letters_list), %timeit will run fewer loops to avoid excessive runtime.

### Checking for membership in a list or tuple is not as efficient!

In [33]:
#Membership of a Dictionary

# identity mapping: 
letters_dict = dict([(x, x) for x in letters_list])

for k, v in letters_dict.items():
    print(k, ":", v)

aaa : aaa
aab : aab
aac : aac
aad : aad
aae : aae
aaf : aaf
aag : aag
aah : aah
aai : aai
aaj : aaj
aak : aak
aal : aal
aam : aam
aan : aan
aao : aao
aap : aap
aaq : aaq
aar : aar
aas : aas
aat : aat
aau : aau
aav : aav
aaw : aaw
aax : aax
aay : aay
aaz : aaz
aba : aba
abb : abb
abc : abc
abd : abd
abe : abe
abf : abf
abg : abg
abh : abh
abi : abi
abj : abj
abk : abk
abl : abl
abm : abm
abn : abn
abo : abo
abp : abp
abq : abq
abr : abr
abs : abs
abt : abt
abu : abu
abv : abv
abw : abw
abx : abx
aby : aby
abz : abz
aca : aca
acb : acb
acc : acc
acd : acd
ace : ace
acf : acf
acg : acg
ach : ach
aci : aci
acj : acj
ack : ack
acl : acl
acm : acm
acn : acn
aco : aco
acp : acp
acq : acq
acr : acr
acs : acs
act : act
acu : acu
acv : acv
acw : acw
acx : acx
acy : acy
acz : acz
ada : ada
adb : adb
adc : adc
add : add
ade : ade
adf : adf
adg : adg
adh : adh
adi : adi
adj : adj
adk : adk
adl : adl
adm : adm
adn : adn
ado : ado
adp : adp
adq : adq
adr : adr
ads : ads
adt : adt
adu : adu
adv : adv


In [34]:
# "aaa" in letters_dict
# "zzz" in letters_dict
%timeit ("aaa" in letters_dict)
%timeit ("mmm" in letters_dict)
%timeit ("zzz" in letters_dict)

35.2 ns ± 3.79 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
30.6 ns ± 0.474 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
32.3 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### You could also convert a list into a set and check for a membership

---

In [39]:
letters_set = set(letters_list)

%timeit ("aaa" in letters_set)
%timeit ("mmm" in letters_set)
%timeit ("zzz" in letters_set)

66.6 ns ± 37.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
40.6 ns ± 6.84 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
41.9 ns ± 5.81 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [62]:
dd = {'a': 1, 'b': 2}
ss = {'a', 'b'}

print('a' in dd, 'b' in ss)


True True


## String Concatenation

•	How is it possible to improve the performance of strings in Python?

- Strings in Python are immutable, which has some advantages and disadvantages. 

- This attribute means that strings can be used as keys in dictionaries and individual copies can be shared among multiple variable bindings. 
- However the disadvantage is that you can’t say something like, “change all the ‘a’s to ‘b’s” in any given string. Instead, you have to create a new string with the desired properties. This continual copying can lead to significant inefficiencies in Python programs because it means using the ‘+’ or ‘+=’ operators may build new strings for each intermediate step.

In [35]:
def make_string(string_list):
    my_string = ''
    for character in string_list:
        my_string += character
    return my_string
str_list = [character for character in 'abcdefghijklmnopqrstuvwxyz']
print(str_list)
print(make_string(str_list))


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
abcdefghijklmnopqrstuvwxyz


In [36]:
%timeit make_string(str_list)

1.29 µs ± 92.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [37]:
%timeit "".join(str_list)

216 ns ± 6.92 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [48]:
','.join(['a', 'b', 'c'])

'a,b,c'

In [51]:
'a,b,c'.split(',')

['a', 'b', 'c']

# Decorator Caching

•	How can I use a decorator to improve the performance of a function?

## Function Decorator



---

- The symbol **@** is Python decorator syntax. 


- Python decorators are callable Python object that is used to modify a function, method or class definition.   


- Python decorators are normally used for tracking, locking, or logging  


- The wise use of decorators can improve the performance of codes.  


- Decorate a Python function so that it remembers the results needed later  

---

In [39]:
def decorating_a_function(func):
    def function_wrapper(x):
        print("Now \"" + func.__name__ + "\" becomes decorated.")
        print("The attribute:")
        func(x)
        print("is surrounded by all these text!")
    return function_wrapper

def foo(x):
    print(x)

In [40]:
# run the function
foo("test")

# separator
print("-" * 50)

# get the function name
print("function_name:", foo.__name__)

test
--------------------------------------------------
function_name: foo


In [3]:
foo = decorating_a_function(foo)

# run the function
foo("hello")

# separator
print("-" * 50)

# get the function name
print(foo.__name__)

Now "foo" becomes decorated.
The attribute:
hello
is surrounded by all these text!
--------------------------------------------------
function_wrapper


In [41]:
def decorating_a_function(func):
    def function_wrapper(x):
        print("Now the function called \"" + func.__name__ + "\" becomes decorated.")
        print("The attribute of the function:")
        func(x)
        print("is surrounded by all this text!")
    return function_wrapper

@decorating_a_function
def foo(x):
    print(x)

In [42]:
# Test 1:
foo("test")

Now the function called "foo" becomes decorated.
The attribute of the function:
test
is surrounded by all this text!


In [6]:
# Test 2:
print(foo.__name__)

function_wrapper


-Wraps: takes a function used in a decorator and add the functionality of copying over the function name, ...

## Imported functions can also be decorated

In [86]:
from math import sin, cos, pi

def our_decorator(func):
    def function_wrapper(x):
        val = func(x)
        print("The result of %s(%0.4f) is: %0.4f" % (func.__name__, x, val))
        return val
    return function_wrapper

# in this case is not possible to use @
sin = our_decorator(sin)
cos = our_decorator(cos)

for f in [sin, cos]: f(pi/2)

for f in [sin, cos]: f(pi)
    


The result of sin(1.5708) is: 1.0000
The result of cos(1.5708) is: 0.0000
The result of sin(3.1416) is: 0.0000
The result of cos(3.1416) is: -1.0000


## Using wraps from module functools

- a module with higher-order functions and operations on callable objects

In [76]:
from functools import wraps

def greeting(func):
    @wraps(func)
    def function_wrapper(x):
        """function_wrapper of greeting"""
        print("Hello, this function " + func.__name__ + " at value " + str(x) + " returns:", func(x))
    
    return function_wrapper

@greeting
def simple_f(x):
    """add 500"""
    return (x + 500)

In [77]:
#call simple_f
simple_f(10)

Hello, this function simple_f at value 10 returns: 510


In [78]:
print("function name: " + simple_f.__name__)
print("docstring: " + simple_f.__doc__)
print("module name: " + simple_f.__module__)

function name: simple_f
docstring: add 500
module name: __main__


-Docstring: is a string used to document  a python module, class, function, or method.

In [7]:
from functools import wraps

def greeting(func):
    def function_wrapper(x):
        """function_wrapper of greeting"""
        print("Hello, this function " + func.__name__ + " at value " + str(x) + " returns:", func(x))
    
    return function_wrapper

@greeting
def simple_f(x):
    """add 500"""
    return (x + 500)

In [9]:
#call simple_f
simple_f(10)

Hello, this function simple_f at value 10 returns: 510


In [8]:
print("function name: " + simple_f.__name__)
print("docstring: " + simple_f.__doc__)
print("module name: " + simple_f.__module__)

function name: function_wrapper
docstring: function_wrapper of greeting
module name: __main__


# Using Decorators for Caching

In [43]:
import time

# Consider Fibonacci numbers: The series of numbers where each number is the sum of two preceding number
# 0, 1,1,2,3,5,8, ...
# defined as f_n = f_{n - 1} + f_{n - 2} for n >=2 
# where f_0 = 0 and f_1 = 1
# https://en.wikipedia.org/wiki/Fibonacci_number
    

# a simple recursion: 
def fib(i):
    if i <= 1: return i
    return fib(i - 1) + fib(i - 2)

#0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...

t = time.process_time()
fib_result = fib(30)
elapsed_time = time.process_time() - t
print("fibonacci time: %0.10f; fib_result: %d" % (elapsed_time, fib_result))

fibonacci time: 0.1562500000; fib_result: 832040


# Memoization

In [14]:
# The idea is a memoization: 
# introduce a map (dictionary) "memo"
# in which to save intermediate steps
# of calculations

def fib_memo(i, memo=dict()):
    if i <= 1: return i
    if i in memo: return memo[i]

    memo[i] = fib_memo(i - 1, memo) + fib_memo(i - 2, memo)
    return memo[i]

t = time.process_time() 
fib_m = fib_memo(30) # 1000
elapsed_time_memo = time.process_time() - t
print("fibonacci time: %0.10f; fib_result: %d" % (elapsed_time_memo, fib_m))

fibonacci time: 0.0000000000; fib_result: 832040


In [29]:
# We can create a decorator that saves:
# each intermediate value in memory 
# rather than calculating it every time.

# @cache: makes things much faster while decreasing the load on computing resources

from functools import wraps

def cache(f):
    memo = {}
    @wraps(f)
    def function_wrapper(*arg):
        if arg not in memo:
            memo[arg] = f(*arg)
        return memo[arg]
    
    return function_wrapper

@cache
def fib_cache(i):
    if i < 2: return i
    return fib_cache(i - 1) + fib_cache(i - 2)


t = time.process_time() 
fib_c = fib_cache(30)
elapsed_time_c = time.process_time() - t
print("fibonacci time: %0.10f; result: %d" % (elapsed_time_c, fib_c))

fibonacci time: 0.0000000000; result: 832040


In [33]:
def add(*numbers):
    return sum(numbers)

print(add(2, 3))
print(add(2, 3, 5))
print(add(2, 3, 5, 7))
print(add(2, 3, 5, 7, 9))

5
10
17
26


# Optimizing Loops

•	What are the main factors affecting the speed of Python loops?

It is important to realize that everything you put in a loop gets executed for every loop iteration. They key to optimizing loops is to minimize what they do. 

In [44]:
import random

lowerlist = ['abcdefghijklmnopqrstuvwxyz'[:random.randint(0, 26)] for x in range(1000)]

# get firs 20 elements
lowerlist[ : 20]

['abcde',
 'abcdefghijklmn',
 'abcd',
 'abcdefghijklmnopq',
 'abcdefgh',
 'abcdefghijkl',
 'abcd',
 'abcdefghijklmnopqrstuv',
 'abcdefghij',
 'abcdefg',
 'abcdefghij',
 'abcdefg',
 'abcdefghijklmnopqrs',
 'abcdefghijklmnopqrst',
 'abcdefghijklmnopqrstuvwxyz',
 'abcdefghi',
 'abcd',
 'abcdef',
 'ab',
 'abcdefghijklmnopqrstuv']

### Task: From the lowerlist build the upperlist

In [17]:
print('hello'.upper())
print(str.upper('hello'))

HELLO
HELLO


In [45]:
#The following code creates a list of the same words in lowerlist but first converts them to uppercase
upperlist = []

def to_upper_1():
    for word in lowerlist:
        upperlist.append(str.upper(word))
    return upperlist

%timeit -n 10000 to_upper_1()

137 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


- The loop calls two methods: "upperlist.append" and "str.upper" every time. 

- Python must support dynamic attributes as well as multiple namespaces. 


In [19]:
# str.upper and upperlist.append into separate variables and 
# replace the calls to these functions with the variable names.
#In this way, we avoid the overhead of searching:

upperlist = []
f_upper = str.upper
f_append = upperlist.append

def to_upper_2():
    for word in lowerlist:
        f_append(f_upper(word))

%timeit -n 1000 to_upper_2()

124 µs ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [145]:
# Another optimization for the loop version is to use local variables rather 
# than global variables as these can be accessed much more efficiently in Python.

def to_upper_3():
    upperlist = []
    f_upper = str.upper
    f_append = upperlist.append

    for word in lowerlist:
        f_append(f_upper(word))

%timeit -n 1000 to_upper_3()

68.7 µs ± 8.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Avoid the loop

In [137]:
# A "map" is often called "apply-to-all" when considered in functional form;
# e.g. a map applied on all elements of a list

# def f(x):
#     return x + 100

simple_mapping = map(lambda x : x + 100, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

list(simple_mapping)

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]

In [138]:
# Apply the method "upper" on strings

print(lowerlist[0], str.upper(lowerlist[0]))

abcdefghijklmnopqrstu ABCDEFGHIJKLMNOPQRSTU


In [148]:

# avoiding the loop by using "map"
upper = str.upper

%timeit -n 1000 upperlist = list(map(str.upper, lowerlist))

40 µs ± 8.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [149]:

# avoiding the loop by using "list comprehension"
f_upper = str.upper 
%timeit -n 1000 upperlist = [f_upper(word) for word in lowerlist]

58.9 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [153]:
# add two random vectors; given as lists

import random 

random_numbers1 = [random.random() for _ in range(0,100000)]
random_numbers2 = [random.random() for _ in range(0,100000)]

%timeit res1 = list(map(lambda x, y: x + y, random_numbers1, random_numbers2))

# However map is calling our function as "adding two vectors" not as "cross product" 

5.75 ms ± 44.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [151]:
len(list(map(lambda x, y: x + y, random_numbers1, random_numbers2)))

100000

## Intrinsic Opertors

- Another performance improvement is to use intrinsic operators (+, -, *, etc.) instead of a user defined function.  


- The operator module exports a set of efficient functions corresponding to the intrinsic operators of Python.   


- operator.add(x, y) is equivalent to the expression x + y.   


- `import operator`

https://docs.python.org/3.4/library/operator.html

In [154]:
import operator

%timeit res2 = list(map(operator.add, random_numbers1, random_numbers2))

2.68 ms ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### using maps

- In Python 3.5, **the map function returns an iterator that does not evaluate the arguments until it needs to**. 

-  By converting the iterator to a list, we are forcing map to compute every value.


In [155]:
#Instead here we use the operator "add" directly without "map":

%timeit res3 = operator.add(random_numbers1, random_numbers2)

411 µs ± 4.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [158]:
# Using numpy: exploit contiguous memory, special instructions

import numpy as np

rand1_np = np.array(random_numbers1)
rand2_np = np.array(random_numbers2)

%timeit res4 = rand1_np + rand2_np

31.2 µs ± 379 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [159]:
# Using Python arrays (if you don't have Numpy...)

import array

rand1_arr = array.array('d', random_numbers1)
rand2_arr = array.array('d', random_numbers2)

%timeit res5 = operator.add(rand1_arr, rand2_arr)

43.9 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
